In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta
from dateutil.parser import parse



In [32]:
COMMA = ','
TODAY = 'Today'
SPACE = ' '

In [151]:
def parse_weekday(date):
    weekday = date.split(' ')[0]
    time = date.split(' ')[2]
    hour = int(time.split(':')[0])
    minute = int(time.split(':')[1])

    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    target_day = days.index(weekday)

    now = datetime.now()

    days_to_next_target_day = (target_day - now.weekday() + 7) % 7
    next_target_day = now + timedelta(days=days_to_next_target_day)

    next_target_day_at_specific_time = next_target_day.replace(hour=hour, minute=minute, second=0, microsecond=0)

    return next_target_day_at_specific_time


def parse_date(date): 
    if COMMA in date: 
        date_object = datetime.strptime(date, "%a, %b %d, %I:%M %p").replace(year=datetime.now().year)
    else: 
        if TODAY in date: 
            hour = int(date.split(' ')[2].split(':')[0])
            minute = int(date.split(' ')[2].split(':')[1])
            date_object = datetime.now().replace(hour=hour, minute=minute, second=0, microsecond=0)
        else:
            date_object = parse_weekday(date)
            
    return date_object

def parse_address(address):
    location = address.find('p', class_='location-info__address-text')
    event_location = location.text
    # remove button and location elements from div with address
    address.find('div', class_='map-button-toggle').decompose()
    location.decompose()
    event_address = address.text.strip()
    return event_address, event_location

def parse_event_date(event_date):
#    Saturday, July 27 · 3 - 11:30pm WEST

    if ' · ' in event_date:
        if COMMA in event_date.split(' · ')[0]:
            event_month = event_date.split(' · ')[0].split(SPACE)[1]
            event_day = event_date.split(' · ')[0].split(SPACE)[2]
        else:
            event_month = event_date.split(' · ')[0].split(SPACE)[0]
            event_day = event_date.split(' · ')[0].split(SPACE)[1]
        time_postfix = None

        event_start_time = event_date.split(' · ')[1].split(' - ')[0]
        if 'am' in event_start_time or 'pm' in event_start_time:
            time_postfix = event_start_time[-2:]
            event_start_time = event_start_time[:-2]
        dt_string = event_month + SPACE + event_day + SPACE + str(datetime.now().year) + SPACE + event_start_time + SPACE + (time_postfix.upper() if time_postfix else "PM")
        if ":" in event_start_time:
            date_object = datetime.strptime(dt_string, '%B %d %Y %I:%M %p')
            print(date_object)
        else:
            date_object = datetime.strptime(dt_string, '%B %d %Y %I %p')
    else:
        date_object = datetime.strptime(event_date.split(' - ')[0], "%a, %d %b %Y %H:%M")

    # event_end_time = event_date.split(' · ')[1].split(' - ')[1].split(SPACE)[0]
    
    return date_object

# todo: change 3 pm to 15:00
# print(parse_event_date('Saturday, July 27 · 3 - 11:30pm WEST'))
# print(parse_event_date('August 1 · 7pm - August 2 · 1am WEST'))
# print(parse_event_date('Fri, 29 Nov 2024 21:00 - Mon, 2 Dec 2024 23:30 WET'))
print(parse_event_date('Thursday, July 18 · 6:30 - 9:30pm EDT'))
# print(parse_date('Sat, Jul 27, 3:00 PM'))
# print(parse_date('Thursday at 4:00 PM'))


2024-07-18 18:30:00
2024-07-18 18:30:00


In [4]:
# todo: think about what to do with pagination
# todo: think about do we need EventCardUrgencySignal__label - going fast, sold out, almost full etc
EVENTBRITE_ID = 2 
EVENTBRITE_URL = 'https://www.eventbrite.com/d/portugal--ilha-da-madeira--85687345/all-events/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(EVENTBRITE_URL, headers=headers)
time.sleep(5)

soup = BeautifulSoup(response.text, 'html.parser')
events = soup.find_all('div', class_='discover-search-desktop-card discover-search-desktop-card--hiddeable')

events


[<div class="discover-search-desktop-card discover-search-desktop-card--hiddeable"><section class="DiscoverHorizontalEventCard-module__cardWrapper___veJo5"><div class="Container_root__4i85v NestedActionContainer_root__1jtfr event-card event-card__horizontal horizontal-event-card__action-visibility" style="--ContainerBgColor:#fff;--ContainerBorderRadius:16px;--ContainerElevationFocusWithin:0px 2px 8px rgba(30, 10, 60, 0.06), 0px 4px 12px rgba(30, 10, 60, 0.08);--ContainerElevationHover:0px 2px 8px rgba(30, 10, 60, 0.06), 0px 4px 12px rgba(30, 10, 60, 0.08);--ContainerPadding:16px"><div data-testid="event-card-tracking-layer" style="position:absolute;top:0px;left:0px;height:100%;pointer-events:none;width:100%"></div><section class="horizontal-event-card__column" style="--HorizontalCardColumnMarginRight:24px"><a aria-label="View THREE PLUS ONE." class="event-card-link" data-event-category="other" data-event-id="872807750477" data-event-location="Funchal, Madeira" data-event-paid-status="p

In [7]:
fields = ["title", "start_date", "end_date", "location", 'address', 'description', 'short_description', 'source_link', 'source_id', 'image_link', 'organizer', 'price', 'last_inserted_at', 'last_updated_at']

data = {field: [] for field in fields}
event_links = []

for event in events: 
    event_card = event.find('a', class_='event-card-link')
    event_link = event_card.get('href')

    event_links.append(event_link)

    # basic_info = event.find('a', class_='event-card-link')
    # data['title'].append(event.find('h2').text)

    # p_elements = event.find_all('p')
    # # remove urgency signal chip data
    # filtered_elements = [p for p in p_elements if 'EventCardUrgencySignal__label' not in p.get('class')]
    # print(filtered_elements[0].text.split(' • ')[0])
    # data['source_link'].append(EVENTBRITE_URL)
    # data['source_id'].append(EVENTBRITE_ID)
    # # todo: add these fields
    # data['description'].append(None)
    # data['short_description'].append(None)
    # data['address'].append(None)

    # # start_date = parse_date(filtered_elements[0].text.split(' • ')[0])
    # # data['start_date'].append(start_date)
    # data['end_date'].append(None)

    # data['image_link'].append(None)
    # data['organizer'].append(None)

    # data['last_inserted_at'].append(datetime.now())
    # data['last_updated_at'].append(datetime.now())
    # # data['time'].append(filtered_elements[0].text.split(' • ')[1])

    # data['location'].append(basic_info.get('data-event-location', '-'))
    # # data['category'].append(basic_info.get('data-event-category', '-'))
    # # todo: solve problem with slow page loading and skeleton insteda of price
    # # price = event.find_all('div', class_='discover-horizontal-event-card__price-wrapper').text


In [113]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service

# from time import sleep
# service = Service(executable_path='/Users/valeryiachyzhova/Desktop/chromedriver-mac-x64/chromedriver')
# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(service=service, options=options)

# driver.get('https://www.eventbrite.com/e/kaue-tickets-940285929527?aff=ebdssbdestsearch')
# sleep(100)

# page_source = driver.page_source
# soup = BeautifulSoup(page_source, 'html.parser')

# print(soup)

# driver.quit()


In [139]:
import json

def get_price(soup):
    # Find all script tags and then find the one containing '__SERVER_DATA__'
    script_tags = soup.find_all('script')
    script_tag = None
    for tag in script_tags:
        if '__SERVER_DATA__' in tag.text:
            script_tag = tag
            break
    if script_tag:
        # Extract the JSON string
        json_str = script_tag.string.split('=', 1)[1].strip()
        # Load the JSON string into a Python dictionary
        data = json.loads(json_str[:-1])
        is_free = data['event_listing_response']['tickets']['ticketClasses'][0]['characteristics']['isFree']
        if is_free:
            minimum_ticket_price_rounded = 'Free'
        else:
            # Access the minimumTicketPriceRounded value
            minimum_ticket_price_rounded = data['event_listing_response']['tickets']['ticketClasses'][0]['cost']['display']
    else:
        minimum_ticket_price_rounded = None
    
    return minimum_ticket_price_rounded


In [161]:
fields = ["title", "start_date", "end_date", "location",  'address', 'description', 'short_description', 'source_link', 'source_id', 'image_link', 'organizer', 'price', 'last_inserted_at', 'last_updated_at']
data = {field: [] for field in fields}


In [162]:
import json

for link in event_links:
    print(link)
    res = requests.get(link, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    event_title = soup.find('h1', class_='event-title').text
    event_price = soup.find('div', class_='conversion-bar__panel-info').text if soup.find('div', class_='conversion-bar__panel-info') else get_price(soup)

    address = soup.find('div', class_='location-info__address')
    event_address, event_location = parse_address(address)

    date = soup.find('span', class_='date-info__full-datetime').text
    event_start_date = parse_event_date(date)
    event_end_date = None

    # todo
    event_description = None
    event_short_description = None

    event_image_link = soup.find('div', class_='event-hero').find('picture').find('img').get('src')

    event_organizer = soup.find('a', class_='descriptive-organizer-info-mobile__name-link').text
    event_source_link = link
    event_source_id = EVENTBRITE_ID
    
    event_last_inserted_at = datetime.now()
    event_last_updated_at = datetime.now()

    data['title'].append(event_title)
    data['address'].append(event_address)
    data['location'].append(event_location)
    data['price'].append(event_price)
    data['start_date'].append(event_start_date)
    data['organizer'].append(event_organizer)

    data['end_date'].append(event_end_date)
    data['description'].append(event_description)
    data['short_description'].append(event_short_description)
    # data['categories'].append(categories)
    data['image_link'].append(event_image_link)

    data['source_link'].append(event_source_link)
    data['source_id'].append(event_source_id)
    data['last_inserted_at'].append(event_last_inserted_at)
    data['last_updated_at'].append(event_last_updated_at)

    print(event_title, event_price, event_address, event_location, event_start_date, event_image_link, event_organizer, event_source_link, event_source_id, event_last_inserted_at, event_last_updated_at)



https://www.eventbrite.pt/e/three-plus-one-tickets-872807750477?aff=ebdssbdestsearch
THREE PLUS ONE. From €22.57 2 Rua Brigadeiro Oudinot 9060-209 Funchal Portugal THREE HOUSE HOTEL 2024-07-27 15:00:00 https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F780880969%2F804704947183%2F1%2Foriginal.20240601-143010?w=600&auto=format%2Ccompress&q=75&sharp=10&rect=0%2C0%2C2160%2C1080&s=d447ebee72b36dc4df2956f3b7149084 Three House Hotel https://www.eventbrite.pt/e/three-plus-one-tickets-872807750477?aff=ebdssbdestsearch 2 2024-07-13 21:00:16.065996 2024-07-13 21:00:16.065998
https://www.eventbrite.com/e/kaue-tickets-940285929527?aff=ebdssbdestsearch
Kaue 15.00 EUR 7 Rua do Favilla 9000-920 Funchal Portugal Trap Music Bar 2024-08-01 19:00:00 https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F801644989%2F648424770443%2F1%2Foriginal.20240704-110645?w=600&auto=format%2Ccompress&q=75&sharp=10&rect=865%2C0%2C1690%2C845&s=92f7366e576b63d1da9822376c5fbebc @kaueevents https://www.event

In [163]:
for key, value in data.items():
    print(f"Length of {key}: {len(value)}")

Length of title: 9
Length of start_date: 9
Length of end_date: 9
Length of location: 9
Length of address: 9
Length of description: 9
Length of short_description: 9
Length of source_link: 9
Length of source_id: 9
Length of image_link: 9
Length of organizer: 9
Length of price: 9
Length of last_inserted_at: 9
Length of last_updated_at: 9


In [164]:
df = pd.DataFrame(data)

df

,title,start_date,end_date,location,address,description,short_description,source_link,source_id,image_link,organizer,price,last_inserted_at,last_updated_at
0,THREE PLUS ONE.,2024-07-27 15:00:00,None,THREE HOUSE HOTEL,2 Rua Brigadeiro Oudinot 9060-209 Funchal Port...,None,None,https://www.eventbrite.pt/e/three-plus-one-tic...,2,https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc....,Three House Hotel,From €22.57,2024-07-13 21:00:16.065996,2024-07-13 21:00:16.065998
1,Kaue,2024-08-01 19:00:00,None,Trap Music Bar,7 Rua do Favilla 9000-920 Funchal Portugal,None,None,https://www.eventbrite.com/e/kaue-tickets-9402...,2,https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc....,@kaueevents,15.00 EUR,2024-07-13 21:00:17.309102,2024-07-13 21:00:17.309103
2,EVAYA,2024-07-19 19:00:00,None,"Rua Doutor Costa Ferreira 6, Funchal, Portugal",6 Rua Doutor Costa Ferreira 9060-077 Funchal P...,None,None,https://www.eventbrite.com/e/evaya-tickets-895...,2,https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc....,Escola Normal,Donation,2024-07-13 21:00:18.109136,2024-07-13 21:00:18.109137
3,´MADEIRADiG 2024,2024-11-29 21:00:00,None,MUDAS - Museu de Arte Contemporânea da Madeira,Estrada Simão Gonçalves Câmara 37 Ilha da Made...,None,None,https://www.eventbrite.co.uk/e/madeiradig-2024...,2,https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc....,MADEIRADiG - Festival for Digital Music & Art,€20,2024-07-13 21:00:19.909929,2024-07-13 21:00:19.909933
4,Concertos L - Tristany Mundu,2024-07-27 22:00:00,None,Estalagem da Ponta do Sol,"Quinta da Rocinha, 06 9360 -529 Ponta do Sol P...",None,None,https://www.eventbrite.com/e/concertos-l-trist...,2,https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc....,Estalagem da Ponta do Sol,Free,2024-07-13 21:00:21.004032,2024-07-13 21:00:21.004034
5,Concertos L - Valter Lobo - Mar Íntimo,2024-07-31 22:00:00,None,Estalagem da Ponta do Sol,"Quinta da Rocinha, 06 9360 -529 Ponta do Sol P...",None,None,https://www.eventbrite.com/e/concertos-l-valte...,2,https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc....,Estalagem da Ponta do Sol,Free,2024-07-13 21:00:22.311237,2024-07-13 21:00:22.314417
6,Community Meditation & Inspiration Sessions,2024-07-16 12:30:00,None,Sangha Cowork Funchal,98 Rua da Casa Branca 9004-535 Funchal Portugal,None,None,https://www.eventbrite.pt/e/community-meditati...,2,https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc....,Sangha Cowork,Free,2024-07-13 21:00:24.176813,2024-07-13 21:00:24.176821
7,Free trial day,2024-07-15 09:30:00,None,Sangha Cowork Funchal,98 Rua da Casa Branca 9004-535 Funchal Portugal,None,None,https://www.eventbrite.pt/e/free-trial-day-tic...,2,https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc....,Sangha Cowork,Free,2024-07-13 21:00:26.224274,2024-07-13 21:00:26.224281
8,Beach REIA (Monthly Real Estate Meeting) Only ...,2024-07-18 18:30:00,None,Hooter's John's Pass,"192 Johns Pass, Boardwalk Pl W, Madeira Beach,...",None,None,https://www.eventbrite.com/e/beach-reia-monthl...,2,https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc....,"Derick O, Mark L, and Rex G",Free,2024-07-13 21:00:27.843710,2024-07-13 21:00:27.843713
